In [1]:
import re
import pickle
from IPython.display import display

import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import FeatureUnion, Pipeline

In [2]:
def convert(name):
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1).lower()

In [3]:
df = pd.read_csv('../input/train_V2.csv')
df.columns = [convert(col) for col in df.columns]

In [4]:
display(df.head(), df.shape, list(df.columns))

,id,group_id,match_id,assists,boosts,damage_dealt,dbn_os,headshot_kills,heals,kill_place,...,revives,ride_distance,road_kills,swim_distance,team_kills,vehicle_destroys,walk_distance,weapons_acquired,win_points,win_place_perc
0,7f96b2f878858a,4d4b580de459be,a10357fd1a4a91,0,0,0.00,0,0,0,60,...,0,0.0000,0,0.00,0,0,244.80,1,1466,0.4444
1,eef90569b9d03c,684d5656442f9e,aeb375fc57110c,0,0,91.47,0,0,0,57,...,0,0.0045,0,11.04,0,0,1434.00,5,0,0.6400
2,1eaf90ac73de72,6a4a42c3245a74,110163d8bb94ae,1,0,68.00,0,0,0,47,...,0,0.0000,0,0.00,0,0,161.80,2,0,0.7755
3,4616d365dd2853,a930a9c79cd721,f1f1f4ef412d7e,0,0,32.90,0,0,0,75,...,0,0.0000,0,0.00,0,0,202.70,3,0,0.1667
4,315c96c26c9aac,de04010b3458dd,6dc8ff871e21e6,0,0,100.00,0,0,0,45,...,0,0.0000,0,0.00,0,0,49.75,2,0,0.1875


(4446966, 29)

['id',
 'group_id',
 'match_id',
 'assists',
 'boosts',
 'damage_dealt',
 'dbn_os',
 'headshot_kills',
 'heals',
 'kill_place',
 'kill_points',
 'kills',
 'kill_streaks',
 'longest_kill',
 'match_duration',
 'match_type',
 'max_place',
 'num_groups',
 'rank_points',
 'revives',
 'ride_distance',
 'road_kills',
 'swim_distance',
 'team_kills',
 'vehicle_destroys',
 'walk_distance',
 'weapons_acquired',
 'win_points',
 'win_place_perc']

Если `SimpleFeatureGenerator` определен в тетрадке, то `n_jobs=-1` в `FeatureUnion` упадет с `pickle error`

In [5]:
# class SimpleFeatureGenerator(BaseEstimator, TransformerMixin):
#     def __init__(self):
#         self.created_features = None
    
#     def transform(self, df):
#         df_features = pd.DataFrame()
#         df_features['total_distance'] = df['ride_distance'] + df['walk_distance'] + df['swim_distance']
        
#         if self.created_features is None:
#             self.created_features = list(df_features.columns)
#         else:
#             assert self.created_features == list(df_features.columns)
#         return df_features

#     def fit(self, x, y=None, **fit_params):
#         return self
    
#     def get_feature_names(self):
#         return self.created_features

In [6]:
from features import SimpleFeatureGenerator

In [7]:
gen = SimpleFeatureGenerator()
gen.fit_transform(df)

,total_distance
0,244.8000
1,1445.0445
2,161.8000
3,202.7000
4,49.7500
5,34.7000
6,13.5000
7,3093.0000
8,799.9000
9,65.6700


In [8]:
pickle.dump(gen, open('todel.pkl', 'wb'))

In [19]:
simple_feature_generator = FeatureUnion(
    n_jobs=-1,
    transformer_list=[
        ('as', SimpleFeatureGenerator()),
        ('fs', SimpleFeatureGenerator()),
        ('gs', SimpleFeatureGenerator()),
        ('hs', SimpleFeatureGenerator()),
    ]
)

In [ ]:
%%time
simple_feature_generator.fit_transform(df)

In [ ]:
simple_feature_generator.get_feature_names()